# THINGS-fMRI usage notes (Modified Version)

### THINGS-fMRI1 b-value extraction notebook

Part of codes are grabbing from the THINGS-data repository: [link](https://github.com/ViCCo-Group/THINGS-data/blob/main/MRI/notebooks/fmri_usage.ipynb)

For a detailed description of the data and the procedures that generated it, see [the THINGS-data preprint](https://doi.org/10.1101/2022.07.22.501123).

In [ ]:
from os.path import join as pjoin
import glob
import numpy as np
import pandas as pd
from nilearn.masking import apply_mask, unmask
from nilearn.plotting import plot_epi, plot_stat_map
from nilearn.image import load_img, index_img, iter_img
import matplotlib.pyplot as plt
import cortex

In [ ]:
# Assumes you've downloaded the THINGS-fMRI data to this directory
basedir = '/Users/yilewang/Desktop'

## Single trial responses

The single trial responses are arguably the easiest way to analyze the THINGS-fMRI data. They contains the magnitude of the fMRI response to each stimulus in each voxel with a single number. The single trial responses are provided in two formats: a) In table format, b) in volumetric format.

### Table format

Besides the fMRI response data, the table format contains metadata about each voxel (such as noise ceilings, pRF parameters, regions of interest) and about the stimulus (such as image file name, trial type, run and session). 

In [ ]:
# Assumes you downloaded the single trial responses in table format to this directory 
betas_csv_dir = pjoin(basedir, 'betas_csv')

# and that you're interested in the data for the first subject
sub = '01'

The `sub-{subject}_ResponseData.h5` files contain the actual single trial responses. Rows are voxels, columns are trials.

In [ ]:
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)  # this may take a minute
print('Single trial response data')
responses.head()

The `sub-{subject}_VoxelMetadata.csv` files contain additional information about each voxel, such as membership to ROIs, reliability measures, and noise ceilings.

In [ ]:
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
voxdata.head()

In [ ]:
print('available voxel metadata:\n', voxdata.columns.to_list())

The voxel indices can be used to reconstruct a volume, e.g. for visualizing results obtained from the single trial responses. Alternatively, the brain mask can be used for that purpose (see below). Membership of each voxel to the available ROIs is dummy coded, e.g. in `voxdata["V1"]` or `voxdata["rFFA"]`. The population receptive field parameters are encoded in the following columns: `prf-eccentricity`, `prf-polarangle`, `prf-size`, and `prf-rsquared`. Finally, different reliability estimates are available in the columns: `nc_testset`, `nc_singletrial`, `splithalf_uncorrected`, and `splithalf_corrected`.

The `sub-{subject}_StimulusMetadata.csv` files contain information about the file name of the image shown in each trial, which run and session a given trial occured in, and the trial_type. 

In [ ]:
# Stimulus metadata
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)
stimdata.head()

> 🚨 **Trial types**
>
> The THINGS-fMRI experiment presented participants with three different trial types:
> - `train`: Participants passively viewed an object image.
> - `test`: Same as train, but these trials belonged to a set of 200 images which were presented in each session. It's main purpose is to allow for estimating the reliability of the single trial responses in a given voxel.
> - `catch`: Participants saw a non-object image and responded with a button press. This was included to ensure participants were engaged throughout the experiment.
>
> Note: Catch trials are excluded from the single trial responses in table format as they are likely not of interest for most applications. However, catch trials are included in the volumetric format in order to make it possible to account for them in analyses.

### Volume format

The single trial responses are also provided in volume format which preserves the spatial structure of the data. The data is broken up into runs and sessions, similar to the raw data files.

In [ ]:
# the directory containing the single trial responses in volume form
betas_vol_dir = pjoin(basedir, 'betas_vol', f'sub-{sub}')
# show directory content
files = glob.glob(pjoin(betas_vol_dir, '*', '*'))
files = [f.replace(basedir, '') for f in files]
print('\n'.join(files[:10]))

The `...betas.nii.gz` files are 3D+time nifti images where the time dimension corresponds to  trials. The `...conditions.tsv` contain the file names of object images for each trial.

In [ ]:
# Load responses for example run
betas_f = pjoin(betas_vol_dir, 'ses-things01', f'sub-{sub}_ses-things01_run-01_betas.nii.gz')
betas_example = load_img(betas_f)

# and plot the volume of the 3rd trial
betas_example = index_img(betas_example, 2)
g = plot_stat_map(
    betas_example, bg_img=None, annotate=False, cmap='twilight', vmax=.4,  draw_cross=False, 
)
g.title('Example: Volumetric single trial response (trial# 3)', bgcolor='white', color='black')

In [ ]:
# load the trial conditions
# Note that the volumetric single trial responses include catch trials
conds_tsv = pjoin(betas_vol_dir, 'ses-things01', f'sub-{sub}_ses-things01_run-01_conditions.tsv')
conds = pd.read_csv(conds_tsv, sep='\t').drop(columns='Unnamed: 0')
print('Content of "...conditions.tsv"')
conds.head()

## Brain masks

The brain masks indicate wether a given voxel is located in the brain or not (`1: brain`, `0: not brain`). They can be used e.g. to create a nifti volume from the results you obtained from the single trial responses.

In [ ]:
# Say you have analyzed the single trial responses in table format have produced these results
# These results are an array of n elements, where n is the number of voxels within the brain.
results = np.random.randn(responses.shape[0])
loinds = voxdata['lLOC'].astype(bool) # pretend we found activity in LOC
results[loinds] += 5

In [ ]:
# you can easily use nilearn's masking functions to transform them 
# to an image object and save it as a nifti file

# This is the provided brain mask
bmask_dir = pjoin(basedir, 'brainmasks')
bmask_f = pjoin(bmask_dir, f'sub-{sub}_space-T1w_brainmask.nii.gz')

# use the brain mask to turn your results array into a 3D image
results_img = unmask(results, bmask_f)
# plot the image to verify
g = plot_stat_map(results_img, bg_img=None, cmap='twilight', draw_cross=False, annotate=False)
# g.add_contours(bmask_f)
g.title('Example: Reconstructing volume from array-like data', bgcolor='white', color='black')
# and save it as an nifti file
results_img.to_filename('results.nii.gz')

> 🚨 **Co-registration and volumetric space**
>
> The THINGS-fMRI data was preprocessed with [fmriprep](https://fmriprep.org/en/stable/), which includes co-registration of all functional images to a high-resolution anatomical MRI image. In other words, all functional data for a given subject (including the brain masks aind regions of interest) was transformed into a common "space", meaning that a given voxel always points to the same location in the brain - with some level of imperfection.
> Of course, it's possible to analyze the data in a different space (e.g. MNI) by downloading the raw data and preprocessing it according to your needs.

## Cortical flat maps

We provide flat maps for each subject for visualizing results on a flat representation of the cortical surface with `pycortex`. Provided that you saved your results as a nifti file in the same space that the volumetric data was in, you can use the flat maps and transformation matrices we prepared. Before you can get started, check out the [pycortex documentation](https://gallantlab.github.io/pycortex/) for an explanation on how to set up your installation.

In [ ]:
# load the results we prepared above as a volume object in pycortex
results_data = np.swapaxes(load_img('results.nii.gz').get_fdata(), 0, -1)
vol_data = cortex.Volume(results_data, 'S1', 'align_auto', cmap='twilight', vmin=-6, vmax=6)
# plot with pycortex
fig = plt.figure(figsize=(8,4))
cortex.quickshow(
    vol_data, pixelwise=True, nanmean=True, colorbar_location='left', with_rois=False, fig=fig,
)
plt.title(f'Example: Visualizing results on flat maps')
plt.show()